In [ ]:
import pandas as pd
import os
import seaborn as sns
from statsmodels.regression.linear_model import OLS, WLS
import statsmodels.regression.linear_model as lm
import statsmodels.stats.diagnostic as smsdia
import statsmodels.stats.outliers_influence as oi
import numpy as np
%matplotlib inline

In [ ]:
bp = pd.read_csv('/Users/hari/Data/Building_Permits_Issued.csv')

In [ ]:
bp.head()

In [ ]:
building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
new_building = building.loc[building['Permit Type Description'].str.contains('New')]

In [ ]:
demolition = bp.loc[bp['Permit Type Description'].str.contains('Demolition')]

In [ ]:
res_demolition = demolition.loc[demolition['Permit Subtype Description'].str.contains('Residential')]

In [ ]:
merged = pd.merge(new_building, res_demolition, how='outer')

In [ ]:
#bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])]

In [ ]:
merged_sf = merged.loc[merged['Permit Subtype Description'].str.contains('Single Family Residence')]

In [ ]:
merged_sf.head()

In [ ]:
new_commercial = ['CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA05R202']

In [ ]:
new_residential = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA04R301', 'CAA05R202', 'CAA08R301', 'CAA10R301', 'CAZ08A001', 'DPLX2CNDO']

In [ ]:
matched = pd.merge(bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])], new_building.loc[new_building['Permit Subtype'].isin(new_commercial + new_residential)], on = 'Address')


In [ ]:
#bp['Permit Type Description'].value_counts()

In [ ]:
#bp['Permit Subtype Description'].value_counts()

In [ ]:
commercialdemo = bp.loc[bp['Permit Subtype']== 'CAZ01A001']

In [ ]:
sf_2009 = pd.read_csv('/Users/hari/Data/2009 single family_final_with_sqft.txt', dtype={'AddressPostalCode': 'str'})
sf_2013 = pd.read_csv('/Users/hari/Data/2013 single family_final_with_sqft.txt',dtype={'AddressPostalCode': 'str'})
sf_2017 = pd.read_csv('/Users/hari/Data/2017 single family_final_with_sqft.txt',dtype={'AddressPostalCode': 'str'})

In [ ]:
sf_2017.loc[sf_2017.AddressFullAddress== '605 SUMMIT OAKS CT']

#building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
print(sf_2009.shape)
print(sf_2009.columns)
print(sf_2009.duplicated().sum())
#sf_2009.loc[sf_2009.duplicated(),:]

In [ ]:
print(sf_2013.shape)
print(sf_2013.columns)
print(sf_2013.duplicated().sum())


In [ ]:
print(sf_2017.shape)
print(sf_2017.duplicated().sum())
sf_2017.sort_values(ascending =False, by ='2017 TOTAL APPR').describe()

In [ ]:
sf_2017.isnull().sum()

In [ ]:
sf1 = pd.merge(sf_2009, sf_2013, on= 'AddressFullAddress', how = 'outer',suffixes = ('_f','_s'))
sf1.columns

sf1.rename(columns = {'FinishedArea_f' : '2009_FinishedArea', 'FinishedArea_s' : '2013_FinishedArea'}, inplace = True)

sf1['APN']= sf1['APN_f'].where(sf1['APN_s'].isnull(), sf1['APN_s'])
sf1['DistrictCode']= sf1['DistrictCode_f'].where(sf1['DistrictCode_s'].isnull(), sf1['DistrictCode_s'])
sf1['Council District']= sf1['Council District_f'].where(sf1['Council District_s'].isnull(), sf1['Council District_s'])
sf1['AddressCity']= sf1['AddressCity_f'].where(sf1['AddressCity_s'].isnull(), sf1['AddressCity_s'])
sf1['AddressPostalCode']= sf1['AddressPostalCode_f'].where(sf1['AddressPostalCode_s'].isnull(), sf1['AddressPostalCode_s'])
#sf1['FinishedArea']= sf1['FinishedArea_f'].where(sf1['FinishedArea_s'].isnull(), sf1['FinishedArea_s'])
sf1.columns

sf1.drop(['APN_f','DistrictCode_f','Council District_f','AddressCity_f',
          'AddressPostalCode_f','DistrictCode_s','Council District_s','AddressCity_s','APN_s','AddressPostalCode_s'], axis = 1,inplace = True)

final_sf = pd.merge(sf1, sf_2017,on= 'AddressFullAddress', how = 'outer', suffixes = ('_m','_n'),indicator = True)
final_sf.columns

final_sf.rename(columns = {'FinishedArea': '2017_FinishedArea'}, inplace = True)

final_sf['APN']= final_sf['APN_m'].where(final_sf['APN_n'].isnull(), final_sf['APN_n'])
final_sf['DistrictCode']= final_sf['DistrictCode_m'].where(final_sf['DistrictCode_n'].isnull(), final_sf['DistrictCode_n'])
final_sf['Council District']= final_sf['Council District_m'].where(final_sf['Council District_n'].isnull(), final_sf['Council District_n'])
final_sf['AddressCity']= final_sf['AddressCity_m'].where(final_sf['AddressCity_n'].isnull(), final_sf['AddressCity_n'])
final_sf['AddressPostalCode']= final_sf['AddressPostalCode_m'].where(final_sf['AddressPostalCode_n'].isnull(), final_sf['AddressPostalCode_n'])
#final_sf['FinishedArea']= final_sf['FinishedArea_m'].where(final_sf['FinishedArea_n'].isnull(), final_sf['FinishedArea_n'])


final_sf.drop(['APN_m','DistrictCode_m','Council District_m','AddressCity_m',
          'AddressPostalCode_m','DistrictCode_n','Council District_n','AddressCity_n','APN_n','AddressPostalCode_n'], axis = 1, inplace = True)
final_sf.columns

final_sf.drop_duplicates(keep ='first', inplace = True)



In [ ]:
final_sf.head()

In [ ]:
final_sf.isnull().sum(axis = 0)
# count_nan = len(final_sf) - final_sf.count()
#final_sf.columns

In [ ]:
final_sf['Pct_Change_2009_2013'] = (final_sf['2013 TOTAL APPR'] - final_sf['2009 TOTAL APPR']) / final_sf['2009 TOTAL APPR']*100
final_sf['Pct_Change_2013_2017'] = (final_sf['2017 TOTAL APPR'] - final_sf['2013 TOTAL APPR']) / final_sf['2013 TOTAL APPR']*100
final_sf['Pct_Change_2009_2017'] = (final_sf['2017 TOTAL APPR'] - final_sf['2009 TOTAL APPR']) / final_sf['2009 TOTAL APPR']*100

In [ ]:
final_sf.shape

In [ ]:
r = 0.05 #Annual interest rate
y = 30 #Length of mortgage (years)
n = 12*y # Number of monthly payments
d = 0.05 # Down payment percentage
final_sf['Mortgage_Per_Year_2009'] = 12* (1-d)*final_sf['2009 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2013'] = 12*(1-d)*final_sf['2013 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2017'] = 12*(1-d)*final_sf['2017 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)

In [ ]:
final_sf.head()

In [ ]:
#final_sf.groupby('Council District').Pct_Change_2013_2017.mean().nlargest(5)

In [ ]:
final_sf.groupby('AddressPostalCode').Pct_Change_2013_2017.mean().nlargest(5)

In [ ]:
final_sf.groupby('Council District').Pct_Change_2009_2017.mean().nlargest(5)

In [ ]:
# number of homes fell in 30% Affordable housing in 2009 based on 6% increase in AMI from 2009 t0 2017
final_sf.loc[final_sf['Mortgage_Per_Year_2009'] < 64900*.6*.3].AddressFullAddress.count()
#aff30.sort_values(ascending = False, by='2009 TOTAL APPR' ).columns

In [ ]:
# number of homes fell in 30% Affordable housing in 2013
final_sf.loc[final_sf['Mortgage_Per_Year_2013'] < 62300*.6*.3].AddressFullAddress.count()

In [ ]:
# number of homes fell in 30% Affordable housing in 2013
final_sf.loc[final_sf['Mortgage_Per_Year_2017'] < 68000*.6*.3].AddressFullAddress.count()

In [ ]:
aff30= final_sf[(final_sf['Mortgage_Per_Year_2009'] < 64900*.6*.3) 
                 & (final_sf['Mortgage_Per_Year_2013'] < 62300*.6*.3) 
                 &(final_sf['Mortgage_Per_Year_2017'] < 68000*.6*.3)]

In [ ]:
aff30.head()

In [ ]:
print(aff30.groupby('Council District').Pct_Change_2009_2013.mean().nlargest(10))
print(aff30.groupby('AddressCity').Pct_Change_2009_2013.mean().nlargest(10))

In [ ]:
from plotly import __version__
print(__version__) # it should be above 1.9
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
# connected = True, plotly connects pandas to interactive java library
import colorlover as cl
init_notebook_mode(connected=True)
print(cf.__version__)
cf.go_offline()

In [ ]:
#aff30.sort_values(ascending = False, by = '2017 TOTAL APPR')

In [ ]:
aff30.loc[aff30['AddressPostalCode']=='37228']

In [ ]:
aff_cds = aff30.groupby('Council District',as_index=False)['2009 TOTAL APPR', '2013 TOTAL APPR','2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'].mean()
aff_cds['Council District'] = pd.to_numeric(aff_cds['Council District'])
aff_cds.sort_values(ascending = False, by = 'Pct_Change_2009_2017')

In [ ]:
aff_cds.describe()

In [ ]:
aff_cds.iplot(kind = 'scatter',x = 'Council District', y = ['Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'],
              xTitle='Council District',yTitle='Pct_Change_2009_2013',title='Percentage Change in affordable Housing in Each council district', ticks='outside')


In [ ]:
aff_cds.iplot(kind='box')

In [ ]:
aff_cds.sort_values(ascending = False, by = 'Pct_Change_2009_2013')

In [ ]:
ryb = cl.scales['11']['div']['RdYlGn']; ryb
color = ['blue','yellow','green','purple', 'red','violet','pink', 'black',
         'lightgrey','orange', 'blue','yellow','Cyan','green','purple', 
         'violet','pink', 'black','lightgrey','orange','red','violet','pink', 
         'black','lightgrey','orange', 'blue','orange','red','violet','pink', 
         'black','lightgrey','orange', 'blue']

In [ ]:
aff_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2009_2013'
              ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2009_2013',
              title='Percentage Change in affordable Housing in Each council district', ticks='outside', color = color)
aff_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2013_2017'
              ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2013_2017',
              title='Percentage Change in affordable Housing in Each council district', ticks='outside',color = color)
aff_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2009_2017'
              ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2009_2017',
              title='Percentage Change in affordable Housing in Each council district', ticks='outside',color = color)

In [ ]:
affcity = aff30.groupby('AddressCity',as_index=False)['2009 TOTAL APPR', '2013 TOTAL APPR','2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'].mean()
#aff2009['AddressPostalCode'] = pd.to_numeric(aff2009['AddressPostalCode'])
affcity.set_index('AddressCity')

In [ ]:
affcity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2009_2013',size = 'Pct_Change_2009_2013',
             xTitle='Council District',yTitle='Pct_Change_2009_2013',
                  title='Percentage Change in affordable Housing 2009-2013 by City', ticks='outside')
affcity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2013_2017',size = 'Pct_Change_2013_2017',
             xTitle='Council District',yTitle='Pct_Change_2013_2017',
                  title='Percentage Change in affordable Housing 2013-2017 by City', ticks='outside')
affcity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2009_2017',size = 'Pct_Change_2009_2017',
             xTitle='Council District',yTitle='Pct_Change_2009_2017',
                  title='Percentage Change in affordable Housing 2009-2017  by City', ticks='outside')


In [ ]:
workforce_aff= final_sf[(final_sf['Mortgage_Per_Year_2009'] < 64900*.6*.3) 
                 & (final_sf['Mortgage_Per_Year_2013'] < 62300*.6*.3) 
                 &(final_sf['Mortgage_Per_Year_2017'] < 68000*.6*.3)]


In [ ]:
tax_2016 = pd.read_csv('/Users/hari/Data/2016tax.csv')
tax_2016.head()

In [ ]:
tax_2016.loc[tax_2016.STATE=='TN']